In [ ]:
# 本文内容翻译从
# Manav Sehgal的titanic-data-science-solutions
# 有兴趣的可以去url看看英文原版 https://www.kaggle.com/startupsci/titanic-data-science-solutions
# 以下就是我们开始入门kaggle的titanic实战代码啦
# 由于本人水平有限，有错的地方欢迎看到的人指出
# 我尽量以一种很通俗易懂的方式让我们明白每一步的意义

In [ ]:
# 首先第一步挂在数据进你的kernel，加载完成后，系统会提示你数据的位置
import pandas as pd
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")
test = pd.read_csv("../input/titanic/test.csv")
train = pd.read_csv("../input/titanic/train.csv")

In [ ]:
# kernel主要的七个步骤内容
# 1.分析问题（预测的问题）
# 2.分析训练数据和测试数据（https://www.kaggle.com/c/titanic/data）
#     着重看我们的变量有哪些，比如关于sex性别这一属性对吧，是男是女很有影响的
#     一般发生这样的事情都是女士优先的，还有age也是很重要的权重属性
# 3.数据预处理
# 4.分析，识别模式，探索应用数据
# 5.开始建模，预测，解决问题（这里是核心的核心）
# 6.可视化，完成报告，提出问题的解决步骤和最后结果
# 7.提交结果，进行竞赛排行(LB)





In [ ]:
#数据分析用到的库
import pandas as pd
import numpy as np
import random as rnd
# 数据可视化用到的库
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#机器学习用到的sklearn库
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



In [ ]:
#载入数据进去(数据的路径在开头处给与)
train_df=pd.read_csv("../input/titanic/train.csv")
test_df=pd.read_csv("../input/titanic/test.csv")
combine=[train_df,test_df]
#combine函数将两个数据相结合 

In [ ]:
#载入进数据之后就要分析数据

#打印特征变量标签
print(train_df.columns.values)

In [ ]:
#首先分析那些特征(feature)是明确的(categorical)
#所谓明确的就是指，不是1就是2的那种特征属性，比如我们的性别，不是男的就是女的
#比如我们的票仓，并不是一等舱就是二等舱，三等舱
#Categorical: Survived, Sex, and Embarked. Ordinal: Pclass.

#再分析哪些特征(feature)是量化的(numerical)
#所谓量化(emm自己编的)就是指，它的取值可以是很多不等，没有具体值的一些属性
#比如我们的年龄，我们的恐惧值，它都是0.00到100.00浮点不值的数值
#Continous: Age, Fare. Discrete: SibSp, Parch.


#预览一部分数据
train_df.head()


In [ ]:
#通过预览图我们可以发现有的数据浮点数很高，比如FARE值，这显然是不利于我们做数据模型的
#通过预览图，我们以可以发现有的诗句是存在错误的

In [ ]:
#我们再来预览尾部数据
#通过尾部预览图我们能从中得出训练数据一共有890个样本
train_df.tail()

In [ ]:
#打印出数据整体信息
#通过整体信息图我们还可以得出数据有缺失的情况存在NaN(NOt A NUMBER)--这些情况我们必须纠正
#试想不纠正的话，做平均值处理的时候会得到一个很错误的结果，毕竟890个样本不变，而你却没有对应那个样本的数据进行求和

train_df.info()
print('_'*40)
test_df.info()

In [ ]:
#从整体表中，我们可以知道Cabin(机舱号)这一特征缺失的特别厉害
#年龄也缺失得很严重

#还需要观察dtype属性(毕竟做数据运算的时候，你不可能用object属性去做加法运算)


In [ ]:
# 接下来我们研究数据的分布情况
# 这点超级重要，能够体现数据的集中属性，帮助我们做一下直观的判断
# 哪些属性对结果也就是(Survive)影响较大
train_df.describe()
# 样本总数为891人，占泰坦尼克号（2,224）旅客实际人数的40％。

# 生存是具有0或1个值的分类特征。

# 约有38％的样本存活下来，代表了32％的实际存活率。

# 大多数乘客（> 75％）没有和父母或孩子一起旅行。

# 将近30％的乘客有兄弟姐妹和/或配偶。

# 票价差异很大，只有极少数乘客（<1％）支付的费用高达512美元。

# 65-80岁年龄段的老年乘客很少（<1％）。

#Ps 这里编者着重说明 统计学符号的含义
# count ：计数的意思
# mean ：求均值(有的属性均值毫无意义，比如乘客ID，担忧的却十分重要比如fare值，存活值，这里可以看出存活率不足0.4)
# std ：标准偏差--度量数据分布的分散程度之标准(代商榷，因为我也不是很了解哈哈哈)
# min :最小值
# 下面都是4分位的那个乘客的特征情况
# (这个有利于做统计学的运算)

In [ ]:
train_df.describe(include=['O'])

In [ ]:
# 接下来就是数据预处理
# 这是一个相当重要的环节，如果数据预处理得好，通过固定的模型算法，得到的结果将会是相当可观的
# 数据预处理的步骤

# 相关性。
# 我们想知道每个功能与生存率的关联程度。 我们希望在项目的早期进行此操作，并将这些快速相关性与项目后期的建模相关性进行匹配。

# 完整性
# 我们可能希望完成“年龄”功能，因为它肯定与生存相关。
# 我们可能要完成“登载”功能，因为它也可能与生存或其他重要功能相关。

# 纠正
# 票证功能可能会从我们的分析中删除，因为它包含很高的重复率（22％），并且票证与生存率之间可能没有关联。
# 客舱功能可能由于高度不完整或在训练和测试数据集中包含许多空值而被删除。
# 旅客ID可能不会从训练数据集中删除，因为它对生存没有帮助。
# 名称功能是相对非标准的，可能不会直接有助于生存，因此可能会被放弃。

# 创造
# 我们可能希望基于Parch和SibSp创建一个称为“家庭”的新功能，以获取船上家庭成员的总数。
# 我们可能要设计名称功能，以将标题提取为新功能。
# 我们可能要为年龄段创建新功能。 这会将连续的数字特征转换为有序的分类特征。
# 如果它有助于我们的分析，我们可能还想创建一个票价范围功能。

# 判别性(一种基于问题明锐思想)
# 我们也可以根据前面提到的问题描述增加假设。
# 女性（性=女性）更有可能存活下来。
# 儿童（年龄<？）存活的可能性更高。
# 上等乘客（Pclass = 1）更有可能幸存下来。

# 相关性分析的时候，我们通过旋转特征进行分析
# 为了确认我们的一些观察和假设，我们可以通过使要素相互对立来快速分析要素相关性。 我们现阶段只能对没有任何空值的要素执行此操作。 仅对分类（Sex），序数（Pclass）或离散（SibSp，Parch）类型的特征这样做也是有意义的。
# Pclass我们观察到Pclass = 1和Survived（分类＃3）之间的显着相关性（> 0.5）。 我们决定在模型中包括此功能。
# 性别我们确认在问题定义期间的观察结果是，性别=女性具有很高的生存率，为74％（分类1）。
# SibSp和Parch这些功能对于某些值具有零相关性。 最好是从这些单个要素中派生一个要素或一组要素（创建＃1）。

train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# 可以看到Pclass(票价等级)对于船员生存来说具有极大相关性


In [ ]:
train_df[['Sex','Survived']].groupby(['Sex'],as_index=False).mean().sort_values(by='Survived',ascending=False) 
#同理


In [ ]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)
#同理

In [ ]:
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
#做完相关性分析后，我们接下来要将相关特征设置合理的特征区间
#我是这样理解的，我们在做模型权重的处理时，需要设置合理的数值，这样会方便我们
#的模型做复杂的，更难的数据处理
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)
#这里可以看出年龄段设置20比较合适

In [ ]:
# grid = sns.FacetGrid(train_df, col='Pclass', hue='Survived')
grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

In [ ]:
#以下分析内容作者就跳过拉哈，将就看吧
#说明下所谓的相关联就是把乘客上的港口的特征数据+男女特征数据相结合
guess_ages = np.zeros((2,3))
guess_ages

In [ ]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

#这里报错 的原因是数据里面含有NAN也就是空值，我觉得这个kernel可以完善一下
#先将NAN值补全

In [ ]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [ ]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df.head()


In [ ]:
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

In [ ]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

In [ ]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

In [ ]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

In [ ]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head(10)

In [ ]:
test_df.head(10)

In [ ]:
#以上内容都是在做预处理最后的结果
#我们得到这样一张非常简明的特征权重图
#   PSID PCLASS SEX AGE FARE EMBARKED TITLE ISALONE AGECLASS
# 1
# 2
# 3
# 4
# 5


In [ ]:
#接下来就是带入机器学习算法，求参数
#这里我们只引入总结
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

#只用线性回归进行预测
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

In [ ]:
#这里得到的正确率是80%

#备注这是一个不成熟的kaggle笔记 笔者有空会回来更改中间的bug处，但是想必大家已经掌握了数据处理的一般流程，接下来跟随笔者进行更难，更复杂的数据处理吧